In [0]:
# Importing Libraries
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import argparse, copy

from model import *
from utils import *
from measures import *

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
device = tf.test.gpu_device_name()
if device != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device))

from google.colab import drive
drive.mount('/content/drive')


%cd /content/drive/My Drive/Colab Notebooks
# pip install -e .

Found GPU at: /device:GPU:0
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks


In [3]:
batch_size = 64
test_batch_size = 1000
epochs = 14
lr = 1e-1
gamma = 0.7
seed = 1
torch.manual_seed(seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

use_cuda = torch.cuda.is_available()
train_loader, test_loader = dataloaders(batch_size, use_cuda)
print (len(train_loader))

model_init = Net().to(device)
# CHANGE AS APPROPRIATE
# optimizer = optim.LBFGS(model.parameters(), lr=lr)
# scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
# saving checkpoints during/after training;;;can save other variables as well
# checkpoint = {'state_dict': model.state_dict(),'optimizer': optimizer.state_dict()}
train_losses, test_losses = [], []

938


In [0]:
# test accuracy problem.....
def main():
    train_size = 60032
    sigmas = [0.001,0.005,0.01,0.05,0.1]      #[0.001, 0.01, 0.1, 1.0] #, for 10 sharpness if very high
    delta = 1e-2
    mean = 0.0
    l_weight_norms = []
    l_sharpness = []
    # var_gradients = []  # maybe with respect to batch size.....
    model = copy.deepcopy(model_init)
    model.to(device)
    optimizer = optim.SGD(model.parameters(), lr=lr)
    #  optimizer = optim.Adagrad(model.parameters(), lr=lr)
    scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
    for epoch in range(1, epochs + 1):
        train_loss = train(model, device, train_loader, optimizer, epoch, batch_size)
        train_losses.append(train_loss)
        test_loss = test(model, device, test_loader, batch_size)
        test_losses.append(test_loss)
        scheduler.step()
        # gradients = get_gradients(model)  # a list of values
        # grads.append(gradients) 
    """run for each sigma"""
    for sigma in sigmas:
        # grads = []
#         resetting model everytime
       
        # variance_score = compute_grad_variance(grads)
        # var_gradients.append(variance_score)

        weight_l2_norm = cal_l2_norm(model)
        weight_bound = compute_bound(model, train_size, sigma, weight_l2_norm, delta)  #### last term 
        l_weight_norms.append(weight_bound)
        print ("norm bound:::", weight_bound)

#       to calculate sharpness, perturb the same model 10 times........then get expected value........
        num_times = 10
        p_train_losses = []
        for iterate in range(num_times):
            p_model = copy.deepcopy(model)
            # p_model.to(device)   # does not make any difference
            # p_optimizer = optim.SGD(p_model.parameters(), lr=lr)
            p_model = weight_pertubation(p_model, mean, sigma, device)
            p_train_loss = test(p_model, device, train_loader, batch_size)
            p_train_losses.append(p_train_loss)
        exp_p_error = sum(p_train_losses) / len(p_train_losses)
        # print ("exp_p_error::", exp_p_error)
        sharpness = exp_p_error - train_loss
        print ("sharpness:::", sharpness)
        l_sharpness.append(sharpness)

        show_losses(train_losses, test_losses)  
    show_sharpness_norm(l_sharpness, l_weight_norms, sigmas)

        
if __name__ == '__main__':
    main()
# Adam, Adagrad about 40% test accuracy

In [0]:
# # test accuracy problem.....
# def main():
#     train_size = 60032
#     sigmas = [0.001,0.005,0.01,0.05,0.1]      #[0.001, 0.01, 0.1, 1.0] #, for 10 sharpness if very high
#     delta = 1e-2
#     mean = 0.0
#     l_weight_norms = []
#     l_sharpness = []
#     var_gradients = []  # maybe with respect to batch size.....
#     """run for each sigma"""
#     for sigma in sigmas:
#         grads = []
# #         resetting model everytime
#         model = copy.deepcopy(model_init)
#         model.to(device)
#         # optimizer = optim.SGD(model.parameters(), lr=lr)
#         optimizer = optim.Adagrad(model.parameters(), lr=lr)
#         scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
#         for epoch in range(1, epochs + 1):
#             train_loss = train(model, device, train_loader, optimizer, epoch, batch_size)
#             train_losses.append(train_loss)
#             test_loss = test(model, device, test_loader, batch_size)
#             test_losses.append(test_loss)
#             scheduler.step()
#             gradients = get_gradients(model)  # a list of values
#             grads.append(gradients) 
#         variance_score = compute_grad_variance(grads)
#         var_gradients.append(variance_score)

#         weight_l2_norm = cal_l2_norm(model)
#         weight_bound = compute_bound(model, train_size, sigma, weight_l2_norm, delta)  #### last term 
#         l_weight_norms.append(weight_bound)
#         print ("norm bound:::", weight_bound)

# #       to calculate sharpness, perturb the same model 10 times........then get expected value........
#         num_times = 10
#         p_train_losses = []
#         for iterate in range(num_times):
#             p_model = copy.deepcopy(model)
#             # p_optimizer = optim.SGD(p_model.parameters(), lr=lr)
#             p_model = weight_pertubation(p_model, mean, sigma, device)
#             p_train_loss = test(p_model, device, train_loader, batch_size)
#             p_train_losses.append(p_train_loss)
#         exp_p_error = sum(p_train_losses) / len(p_train_losses)
#         print ("exp_p_error::", exp_p_error)
#         sharpness = exp_p_error - train_loss
#         l_sharpness.append(sharpness)

#         show_losses(train_losses, test_losses)  
#     show_sharpness_norm(l_sharpness, l_weight_norms, sigmas)

        
# if __name__ == '__main__':
#     main()

# # we can vary many things............
# # RMSProp is working shot    
# # Adam, Adagrad about 40% test accuracy

In [0]:
# norms = [297.2753, 29.7270, 2.9733, 0.3045]
# norms = torch.tensor(norms)
# norms = (norms - torch.mean(norms)) / norms.std()
# print (norms)
# exp_p_error = [0.316660638815209, 0.29393473547905175, 1.9219535219771013, 591.3643927659052]
# error = torch.tensor(exp_p_error)
# error = (error - torch.mean(error)) / error.std()
# print (error)
# # norm --- it is a tensor...
# sigmas = [0.001, 0.01, 0.1, 1.0] 
# show_sharpness_norm(error, norms, sigmas)